In [2]:
%matplotlib notebook

In [1]:
import numpy as np
from facenet_models import FacenetModel
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.pyplot import text
import skimage.io as io
import cv2

In [ ]:
def find_eyes(image_data, *, box_threshold=0.97, prop_const=0.13):
    """
    Displays an image with boxes around people's faces and labels them with names.
    Parameters
    ----------
    image_data : numpy.ndarray, shape-(R, C, 3) (RGB is the last dimension)
        Pixel information for the image.
    """
    i = 0
    leftBox = []
    rightBox = []
    
    while i < 4:
        model = FacenetModel()
        
        # in the future, only analyze the highest probability face?
        boxes, probabilities, landmarks = model.detect(image_data)
        face_detected = True

        if boxes is None or probabilities[0]<box_threshold:
            i+=1
            image_data = cv.rotate(image_data, cv.ROTATE_90_CLOCKWISE)
            continue
        
        box = boxes[0]
        prob = probabilities[0]
        
        lefteye = landmarks[0][0]
        righteye = landmarks[0][1]

        # boxes in form [left,top,right,bottom]

        radius = ((box[3] - box[1]) + (box[2] - box[0]))/2 * prop_const

        leftBox = np.array([lefteye[0] - radius, 
                            lefteye[1] - radius, 
                            lefteye[0] + radius,
                            lefteye[1] + radius])
        rightBox = np.array([righteye[0] - radius, 
                            righteye[1] - radius, 
                            righteye[0] + radius,
                            righteye[1] + radius])

        leftBox = np.round(np.array(leftBox)).astype(int)
        rightBox = np.round(np.array(rightBox)).astype(int)
        
        break

    if len(leftBox)>0:
        fig, ax = plt.subplots()
        ax.imshow(image_data)

        ax.add_patch(Rectangle(leftBox[:2], *(leftBox[2:] - leftBox[:2]), fill=None, lw=2, color="yellow"))
        ax.add_patch(Rectangle(rightBox[:2], *(rightBox[2:] - rightBox[:2]), fill=None, lw=2, color="yellow"))
        
        ax.plot(lefteye[0], lefteye[1], "+", color="blue")
        ax.plot(righteye[0], righteye[1], "+", color="blue")
    
    return leftBox, rightBox, image_data

In [ ]:
def eye_img(filepath):
    # pic = np.array(Image.open(filepath))[:,:,:3]
    # shape-(Height, Width, Color)
    pic = io.imread(str(filepath))

    if pic.shape[-1] == 4:
        # Image is RGBA, where A is alpha -> transparency
        # Must make image RGB.
        pic = pic[..., :-1]  # png -> RGB

    left, right, pic = find_eyes(pic)

    if len(left)==0:
        print("FACE NOT FOUND")
        return None

    
    gray = cv2.cvtColor(pic, cv2.COLOR_RGB2GRAY)
    
    # boxes in form [left,top,right,bottom]
    fig, axes = plt.subplots(1,2)

    lefteye = gray[left[1]:left[3], left[0]:left[2]]
    righteye = gray[right[1]:right[3], right[0]:right[2]]
    
    lefteye = cv2.resize(lefteye, (24,24), interpolation = cv2.INTER_LINEAR)
    righteye = cv2.resize(righteye, (24,24), interpolation = cv2.INTER_LINEAR)

    axes[0].imshow(lefteye, cmap="gray")
    axes[1].imshow(righteye, cmap="gray")
    
    return lefteye, righteye
eyedata = eye_img()

In [ ]:
a = np.empty((1,2,2), dtype=int)
b = np.array([[[3,4],[5,6]]])
print(np.concatenate((a,b)))

In [1]:
import numpy as np
from facenet_models import FacenetModel
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.pyplot import text
import skimage.io as io
import cv2
import glob

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
closed_x = np.empty((1,24,24))
open_x = np.empty((1,24,24))

for filename in glob.glob('train_dataset/closed*/*.*'):
    img = np.array([cv2.imread(filename, 0)])/255 + 1

    closed_x = np.concatenate((closed_x, img))
    # 	closed_x.append(img)


for filename in glob.glob('train_dataset/open*/*.*'):
    img = np.array([cv2.imread(filename, 0)])/255 + 1

    open_x = np.concatenate((open_x, img))
    # 	open_x.append(img)

# closed_x = np.array(closed_x)
# open_x = np.array(open_x)

#print(closed_x)

closed_x = closed_x[1:]
open_x = open_x[1:]

In [5]:
train_x = np.vstack((closed_x[:int(len(closed_x)*0.8)], open_x[:int(len(open_x)*0.8)]))
valid_x = np.vstack((closed_x[int(len(closed_x)*0.8):], open_x[int(len(open_x)*0.8):]))

train_y = [0 for i in range(int(len(closed_x)*0.8))] + [1 for i in range(int(len(open_x)*0.8))]
valid_y = [0 for i in range(len(closed_x) - int(len(closed_x)*0.8))] + [1 for i in range(len(open_x) - int(len(open_x)*0.8))]

train_x = torch.tensor(train_x)
valid_x = torch.tensor(valid_x)

train_y = torch.tensor(train_y)
valid_y = torch.tensor(valid_y)

print(train_x)

tensor([[[1.6863, 1.6745, 1.6471,  ..., 1.4549, 1.4941, 1.5216],
         [1.5961, 1.5412, 1.4745,  ..., 1.3294, 1.3373, 1.3451],
         [1.5216, 1.4353, 1.3412,  ..., 1.4000, 1.3686, 1.3490],
         ...,
         [1.5373, 1.5294, 1.5373,  ..., 1.6627, 1.7098, 1.6863],
         [1.6863, 1.6588, 1.6353,  ..., 1.6235, 1.7059, 1.6902],
         [1.6039, 1.5922, 1.6078,  ..., 1.5882, 1.6863, 1.6902]],

        [[1.2000, 1.1961, 1.1686,  ..., 1.2863, 1.3176, 1.3412],
         [1.2000, 1.1961, 1.1725,  ..., 1.2392, 1.2667, 1.2863],
         [1.1882, 1.1843, 1.1647,  ..., 1.2235, 1.2471, 1.2627],
         ...,
         [1.4941, 1.4941, 1.4902,  ..., 1.5412, 1.5216, 1.5098],
         [1.5098, 1.4902, 1.4745,  ..., 1.5176, 1.4980, 1.4824],
         [1.5216, 1.4941, 1.4667,  ..., 1.4902, 1.4667, 1.4510]],

        [[1.5686, 1.5922, 1.6118,  ..., 1.6941, 1.6902, 1.6863],
         [1.6275, 1.6549, 1.6667,  ..., 1.7255, 1.7059, 1.6941],
         [1.6118, 1.6353, 1.6431,  ..., 1.7137, 1.7020, 1.

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, numpydata, numpylabels, transform=None, target_transform=None):
        self.imgs = numpydata
        self.img_labels = numpylabels
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        label = self.img_labels[idx]
        image = self.imgs[idx]
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [6]:
class EyeModel(nn.Module):
    def __init__(self):
        super().__init__()
        # assumes 32x32
        self.conv1 = nn.Conv2d(1, 1, 3, bias=False)
        torch.nn.init.normal_(self.conv1.weight)
        
        self.pool = nn.MaxPool2d(2, 2)
        #self.conv2 = nn.Conv2d(24, 48, 3)
        self.fc1 = nn.Linear(484, 2, bias=False)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x

In [8]:
model = EyeModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 1
batch_size = 1

data = CustomImageDataset(train_x, train_y)
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    #print([i for i in enumerate(trainloader,0)])
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.reshape(batch_size,1,24,24).float()
        
        # zero the parameter gradients
        optimizer.zero_grad()
        print(optimizer)

        # forward + backward + optimize
        input("1")
        outputs = model(inputs)
        print(outputs)
        
        input("2")
        loss = criterion(outputs, labels)
        
        for i in model.parameters():
            print(i.data)
            print("\n\n")
        
        input("3")
        loss.backward()
        input("4")
        optimizer.step()
        # print statistics
        #running_loss += loss.item()
        #if i % 50 == 49:    # print every 2000 mini-batches
        #    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
        #    running_loss = 0.0
        print(i)
    print(epoch)
print('Finished Training')

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)
1


[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


tensor([[ 6.9654, -1.1102]], grad_fn=<MmBackward0>)
2
tensor([[[[-0.0238, -0.1189,  1.2391],
          [-0.5415,  1.0448,  2.1774],
          [ 0.6798,  0.1673,  1.7321]]]])



tensor([[ 4.4120e-02,  3.3957e-02,  1.8574e-02,  4.2630e-02,  2.4977e-02,
         -5.5095e-03, -3.8430e-02,  3.1775e-02,  4.4727e-02,  3.8510e-02,
          1.6511e-02,  3.9587e-02,  3.6026e-02,  3.6010e-02, -2.2977e-02,
         -1.7371e-02, -2.8492e-02,  3.5740e-02, -2.6902e-02,  1.7343e-02,
         -6.8984e-03, -1.3281e-03, -2.7108e-02,  2.0522e-02, -1.7738e-02,
         -3.3459e-02,  1.2200e-02,  4.1041e-02,  4.2453e-02,  1.8991e-03,
         -2.2464e-02,  3.1991e-02,  2.0484e-02,  9.8831e-03,  4.4030e-03,
         -2.3901e-03, -5.6453e-03,  9.2949e-03, -3.5332e-02,  2.1663e-02,
          3.0366e-03,  3.2123e-02,  3.3377e-04,  2.8598e-02,  4.2124e-02,
          4.4840e-02,  2.9834e-02, -8.3366e-03, -3.2100e-02, -3.4980e-02,
         -3.4148e-02,  1.3688e-02,  1.1916e-02,  6.5685e-03, -4.3432e-02,
         

3
4
Parameter containing:
tensor([[ 4.3120e-02,  3.2957e-02,  1.7574e-02,  4.1630e-02,  2.3977e-02,
         -6.5095e-03, -3.9430e-02,  3.0775e-02,  4.3727e-02,  3.7510e-02,
          1.5511e-02,  3.8587e-02,  3.5026e-02,  3.5010e-02, -2.3977e-02,
         -1.8371e-02, -2.9492e-02,  3.4740e-02, -2.7902e-02,  1.6343e-02,
         -7.8984e-03, -2.3281e-03, -2.8108e-02,  1.9522e-02, -1.8738e-02,
         -3.4459e-02,  1.1200e-02,  4.0041e-02,  4.1453e-02,  8.9909e-04,
         -2.3464e-02,  3.0991e-02,  1.9484e-02,  8.8831e-03,  3.4030e-03,
         -3.3901e-03, -6.6453e-03,  8.2949e-03, -3.6332e-02,  2.0663e-02,
          2.0366e-03,  3.1123e-02, -6.6623e-04,  2.7598e-02,  4.1124e-02,
          4.3840e-02,  2.8834e-02, -9.3366e-03, -3.3100e-02, -3.5980e-02,
         -3.5148e-02,  1.2688e-02,  1.0916e-02,  5.5685e-03, -4.4432e-02,
         -9.6458e-04,  2.5003e-02,  3.3961e-03, -4.8134e-03, -3.1925e-02,
         -3.0857e-02, -1.9507e-02, -3.2915e-02,  2.7812e-02,  1.3638e-02,
          3.

1
tensor([[2.3409, 3.9910]], grad_fn=<MmBackward0>)
2
tensor([[[[-0.0248, -0.1199,  1.2381],
          [-0.5425,  1.0438,  2.1764],
          [ 0.6788,  0.1663,  1.7311]]]])



tensor([[ 4.3120e-02,  3.2957e-02,  1.7574e-02,  4.1630e-02,  2.3977e-02,
         -6.5095e-03, -3.9430e-02,  3.0775e-02,  4.3727e-02,  3.7510e-02,
          1.5511e-02,  3.8587e-02,  3.5026e-02,  3.5010e-02, -2.3977e-02,
         -1.8371e-02, -2.9492e-02,  3.4740e-02, -2.7902e-02,  1.6343e-02,
         -7.8984e-03, -2.3281e-03, -2.8108e-02,  1.9522e-02, -1.8738e-02,
         -3.4459e-02,  1.1200e-02,  4.0041e-02,  4.1453e-02,  8.9909e-04,
         -2.3464e-02,  3.0991e-02,  1.9484e-02,  8.8831e-03,  3.4030e-03,
         -3.3901e-03, -6.6453e-03,  8.2949e-03, -3.6332e-02,  2.0663e-02,
          2.0366e-03,  3.1123e-02, -6.6623e-04,  2.7598e-02,  4.1124e-02,
          4.3840e-02,  2.8834e-02, -9.3366e-03, -3.3100e-02, -3.5980e-02,
         -3.5148e-02,  1.2688e-02,  1.0916e-02,  5.5685e-03, -4.4432e-02,
         

3
4
Parameter containing:
tensor([[ 4.2350e-02,  3.2186e-02,  1.6802e-02,  4.0859e-02,  2.3207e-02,
         -7.2788e-03, -4.0199e-02,  3.0005e-02,  4.2952e-02,  3.6735e-02,
          1.4735e-02,  3.7808e-02,  3.4247e-02,  3.4231e-02, -2.4755e-02,
         -1.9147e-02, -3.0266e-02,  3.3969e-02, -2.8670e-02,  1.5577e-02,
         -8.6642e-03, -3.0953e-03, -2.8876e-02,  1.8754e-02, -1.9507e-02,
         -3.5228e-02,  1.0431e-02,  3.9272e-02,  4.0682e-02,  1.2521e-04,
         -2.4245e-02,  3.0207e-02,  1.8697e-02,  8.0925e-03,  2.6141e-03,
         -4.1790e-03, -7.4316e-03,  7.5112e-03, -3.7111e-02,  1.9887e-02,
          1.2648e-03,  3.0354e-02, -1.4348e-03,  2.6829e-02,  4.0360e-02,
          4.3075e-02,  2.8065e-02, -1.0108e-02, -3.3872e-02, -3.6753e-02,
         -3.5924e-02,  1.1909e-02,  1.0131e-02,  4.7794e-03, -4.5224e-02,
         -1.7599e-03,  2.4210e-02,  2.6044e-03, -5.6019e-03, -3.2711e-02,
         -3.1640e-02, -2.0285e-02, -3.3689e-02,  2.7041e-02,  1.2869e-02,
          3.

KeyboardInterrupt: Interrupted by user

In [ ]:
a=torch.tensor([[1.0,2.4]])
b=torch.tensor([0])

criterion(a,b)

In [ ]:
class TModel(nn.Module):
    def __init__(self):
        super().__init__()
        # assumes 32x32
        self.fc1 = nn.Linear(1, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x

test = TModel()
opt = optim.Adam(test.parameters(), lr=0.01)

for i in range(50):
    out = test(torch.tensor([[5]]).float())
    out.backward()
    opt.step()

    print(test.fc1.weight)